In [1]:
%matplotlib inline
# !pip install stargazer
# !pip install scipy
# !pip install plotly_express
# !pip install pandas-profiling
# !pip install linearmodels
# !pip install sklearn

import scipy
import numpy as np
import pandas as pd
import pandas.io.formats.style
import seaborn as sns
import statsmodels as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from IPython.display import HTML
import linearmodels as lm
from linearmodels import PanelOLS
import warnings
warnings.filterwarnings("ignore")
import math
# import sklearn

from linearmodels.panel import compare
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm_api


In [2]:
# path="/Users/mengxi/Desktop/financial research/github final/"
# data=pd.read_stata(path+"211231_total_and_by_industry_data_county.dta")


path="/Users/effie/Desktop/term3/research_module/data/"
data=pd.read_stata(path+"county_clean/total_and_by_industry_data_county.dta")
data=data.drop(columns=['index'])


data


,year,Gebietseinheit,EU_Code,Regional_key,Land,resident,ln_gdp,gdp_level,gdp_per_inhabitant,gdp_per_employee,...,toas_ratio,emp_agr_weight,emp_man_weight,emp_ser_weight,gva_agr_weight,gva_man_weight,gva_ser_weight,vow_agr_weight,vow_man_weight,vow_ser_weight
0,2004,"Stuttgart, Landeshauptstadt, Stadtkreis",DE111,08111,BW,573.955017,-1.059015,40680.228,70877.0,85811.0,...,1.0,0.002413,0.213166,0.784421,0.000634,0.324826,0.674540,0.002775,0.223713,0.773512
1,2004,"Böblingen, Landkreis",DE112,08115,BW,366.351990,-3.269339,14478.332,39520.0,67825.0,...,1.0,0.004872,0.422414,0.572714,0.001776,0.505710,0.492514,0.006840,0.446232,0.546928
2,2004,"Esslingen, Landkreis",DE113,08116,BW,503.649994,-0.074150,15205.272,30190.0,61149.0,...,1.0,0.006507,0.376005,0.617488,0.002375,0.408458,0.589167,0.008329,0.397011,0.594661
3,2004,"Göppingen, Landkreis",DE114,08117,BW,255.177994,-0.930964,6158.588,24134.0,54533.0,...,1.0,0.013096,0.400742,0.586162,0.006225,0.432973,0.560802,0.019670,0.424832,0.555498
4,2004,"Ludwigsburg, Landkreis",DE115,08118,BW,504.459991,0.341932,15851.325,31422.0,67176.0,...,1.0,0.014722,0.341771,0.643507,0.005007,0.376126,0.618868,0.018984,0.361308,0.619708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3586,2012,Saale-Orla-Kreis,DEG0K,16075,TH,84.847000,5.540099,1955.848,23051.0,48013.0,...,NaN,0.044653,0.428515,0.526831,0.031515,0.474068,0.494417,NaN,NaN,NaN
3587,2012,"Greiz, Landkreis",DEG0L,16076,TH,103.880997,2.040082,1854.354,17851.0,46776.0,...,0.0,0.037358,0.373862,0.588780,0.031252,0.365997,0.602751,NaN,NaN,NaN
3588,2012,"Altenburger Land, Landkreis",DEG0M,16077,TH,95.289001,-1.616072,1689.681,17732.0,45168.0,...,1.0,0.028015,0.338902,0.633083,0.023566,0.340479,0.635954,NaN,NaN,NaN
3589,2012,"Eisenach, Kreisfreie Stadt",DEG0N,16056,TH,NaN,NaN,NaN,NaN,NaN,...,0.0,0.002846,0.296681,0.700473,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data["No_ratio"].describe()

count    2691.000000
mean        0.286676
std         0.340037
min         0.000000
25%         0.000000
50%         0.166667
75%         0.500000
max         1.000000
Name: No_ratio, dtype: float64

In [4]:
data["toas_ratio"].describe()

count    2691.000000
mean        0.625744
std         0.476284
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.146924
Name: toas_ratio, dtype: float64

In [5]:
query_data = data.query('No_ratio>0.286')
data.loc[:,'county_affected_no'] = data.index.map(lambda x : 1 if x in query_data.index else 0)

query_data = data.query('toas_ratio>0.627')
data.loc[:,'county_affected_toas'] = data.index.map(lambda x : 1 if x in query_data.index else 0)

query_data = data.query('year>2008')
data.loc[:,'year_post'] = data.index.map(lambda x : 1 if x in query_data.index else 0)
data

,year,Gebietseinheit,EU_Code,Regional_key,Land,resident,ln_gdp,gdp_level,gdp_per_inhabitant,gdp_per_employee,...,emp_ser_weight,gva_agr_weight,gva_man_weight,gva_ser_weight,vow_agr_weight,vow_man_weight,vow_ser_weight,county_affected_no,county_affected_toas,year_post
0,2004,"Stuttgart, Landeshauptstadt, Stadtkreis",DE111,08111,BW,573.955017,-1.059015,40680.228,70877.0,85811.0,...,0.784421,0.000634,0.324826,0.674540,0.002775,0.223713,0.773512,1,1,0
1,2004,"Böblingen, Landkreis",DE112,08115,BW,366.351990,-3.269339,14478.332,39520.0,67825.0,...,0.572714,0.001776,0.505710,0.492514,0.006840,0.446232,0.546928,0,1,0
2,2004,"Esslingen, Landkreis",DE113,08116,BW,503.649994,-0.074150,15205.272,30190.0,61149.0,...,0.617488,0.002375,0.408458,0.589167,0.008329,0.397011,0.594661,0,1,0
3,2004,"Göppingen, Landkreis",DE114,08117,BW,255.177994,-0.930964,6158.588,24134.0,54533.0,...,0.586162,0.006225,0.432973,0.560802,0.019670,0.424832,0.555498,1,1,0
4,2004,"Ludwigsburg, Landkreis",DE115,08118,BW,504.459991,0.341932,15851.325,31422.0,67176.0,...,0.643507,0.005007,0.376126,0.618868,0.018984,0.361308,0.619708,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3586,2012,Saale-Orla-Kreis,DEG0K,16075,TH,84.847000,5.540099,1955.848,23051.0,48013.0,...,0.526831,0.031515,0.474068,0.494417,NaN,NaN,NaN,0,0,1
3587,2012,"Greiz, Landkreis",DEG0L,16076,TH,103.880997,2.040082,1854.354,17851.0,46776.0,...,0.588780,0.031252,0.365997,0.602751,NaN,NaN,NaN,0,0,1
3588,2012,"Altenburger Land, Landkreis",DEG0M,16077,TH,95.289001,-1.616072,1689.681,17732.0,45168.0,...,0.633083,0.023566,0.340479,0.635954,NaN,NaN,NaN,1,1,1
3589,2012,"Eisenach, Kreisfreie Stadt",DEG0N,16056,TH,NaN,NaN,NaN,NaN,NaN,...,0.700473,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1


In [6]:
data.columns

Index(['year', 'Gebietseinheit', 'EU_Code', 'Regional_key', 'Land', 'resident',
       'ln_gdp', 'gdp_level', 'gdp_per_inhabitant', 'gdp_per_employee',
       'gdp_per_hour_per_employed', 'ln_gva', 'gva_agr', 'gva_man', 'gva_ser',
       'gva_ser_fin', 'gva_ser_trade', 'gva_ser_public', 'employee', 'emp_agr',
       'emp_man', 'emp_ser', 'emp_ser_fin', 'emp_ser_trade', 'emp_ser_public',
       'volume_of_work', 'vow_agr', 'vow_man', 'vow_ser', 'vow_ser_fin',
       'vow_ser_trade', 'vow_ser_public', 'firms_no', 'firms_aff_no',
       'No_ratio', 'toas_sum_county', 'toas_aff_sum_county', 'toas_ratio',
       'emp_agr_weight', 'emp_man_weight', 'emp_ser_weight', 'gva_agr_weight',
       'gva_man_weight', 'gva_ser_weight', 'vow_agr_weight', 'vow_man_weight',
       'vow_ser_weight', 'county_affected_no', 'county_affected_toas',
       'year_post'],
      dtype='object')

In [7]:
data["affected_after_no"] = data.county_affected_no*data.year_post
data["affected_after_toas"] = data.county_affected_toas*data.year_post
data["ln_res"]=np.log(data["resident"]*1000)

data["ln_gdp_level"] = np.log(data.gdp_level*1000000)
data["ln_gdp_per_inhabitant"] = np.log(data.gdp_per_inhabitant)
data["ln_gdp_per_employee"] = np.log(data.gdp_per_employee)

data["ln_gva_agr"] = np.log(data.gva_agr*1000000)
data["ln_gva_man"] = np.log(data.gva_man*1000000)
data["ln_gva_ser"] = np.log(data.gva_ser*1000000)
data["ln_gva_ser_fin"] = np.log(data.gva_ser_fin*1000000)
data["ln_gva_ser_trade"] = np.log(data.gva_ser_trade*1000000)
data["ln_gva_ser_public"] = np.log(data.gva_ser_public*1000000)

data["ln_emp"]=np.log(data["employee"]*1000)

data["ln_emp_ser_fin"] = np.log(data.emp_ser_fin*1000)
data["ln_emp_ser_trade"] = np.log(data.emp_ser_trade*1000)
data["ln_emp_ser_public"] = np.log(data.emp_ser_public*1000)
data["ln_emp_agr"]=np.log(data["emp_agr"]*1000)
data["ln_emp_man"]=np.log(data["emp_man"]*1000)
data["ln_emp_ser"]=np.log(data["emp_ser"]*1000)

data["ln_vow"]=np.log(data["volume_of_work"]*1000000)

data["ln_vow_ser_fin"] = np.log(data.vow_ser_fin*1000000)
data["ln_vow_ser_trade"] = np.log(data.vow_ser_trade*1000000)
data["ln_vow_ser_public"] = np.log(data.vow_ser_public*1000000)
data["ln_vow_agr"]=np.log(data["vow_agr"]*1000000)
data["ln_vow_man"]=np.log(data["vow_man"]*1000000)
data["ln_vow_ser"]=np.log(data["vow_ser"]*1000000)

df_fe = data.set_index(['Regional_key','year'])
df_fe

,,Gebietseinheit,EU_Code,Land,resident,ln_gdp,gdp_level,gdp_per_inhabitant,gdp_per_employee,gdp_per_hour_per_employed,ln_gva,...,ln_emp_agr,ln_emp_man,ln_emp_ser,ln_vow,ln_vow_ser_fin,ln_vow_ser_trade,ln_vow_ser_public,ln_vow_agr,ln_vow_man,ln_vow_ser
Regional_key,year,,,,,,,,,,,,,,,,,,,,,
08111,2004,"Stuttgart, Landeshauptstadt, Stadtkreis",DE111,BW,573.955017,-1.059015,40680.228,70877.0,85811.0,58.340000,-0.712461,...,7.042286,11.523420,12.826294,20.362797,19.048334,18.897976,19.067344,14.475618,18.865405,20.105983
08115,2004,"Böblingen, Landkreis",DE112,BW,366.351990,-3.269339,14478.332,39520.0,67825.0,47.110001,-2.930522,...,6.946976,11.409463,11.713864,19.543338,17.615193,18.162907,17.646475,14.558400,18.736418,18.939897
08116,2004,"Esslingen, Landkreis",DE113,BW,503.649994,-0.074150,15205.272,30190.0,61149.0,42.700001,0.275855,...,7.388946,11.445685,11.941742,19.690759,17.683749,18.393741,18.013735,14.902725,18.766969,19.170997
08117,2004,"Göppingen, Landkreis",DE114,BW,255.177994,-0.930964,6158.588,24134.0,54533.0,38.419998,-0.583964,...,7.299121,10.720113,11.100390,18.892520,16.678454,17.398001,17.386957,14.963865,18.036458,18.304630
08118,2004,"Ludwigsburg, Landkreis",DE115,BW,504.459991,0.341932,15851.325,31422.0,67176.0,47.139999,0.693390,...,8.153062,11.297837,11.930629,19.633333,17.833100,18.261110,18.028408,15.669149,18.615310,19.154826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16075,2012,Saale-Orla-Kreis,DEG0K,TH,84.847000,5.540099,1955.848,23051.0,48013.0,NaN,5.593267,...,7.506042,9.767439,9.973993,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16076,2012,"Greiz, Landkreis",DEG0L,TH,103.880997,2.040082,1854.354,17851.0,46776.0,NaN,2.091659,...,7.300473,9.603800,10.057967,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16077,2012,"Altenburger Land, Landkreis",DEG0M,TH,95.289001,-1.616072,1689.681,17732.0,45168.0,NaN,-1.566608,...,6.954639,9.447623,10.072513,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
data_check = df_fe[['emp_agr_weight','emp_man_weight','emp_ser_weight','gdp_level','gdp_per_inhabitant','gdp_per_hour_per_employed']]
data_check

,,emp_agr_weight,emp_man_weight,emp_ser_weight,gdp_level,gdp_per_inhabitant,gdp_per_hour_per_employed
Regional_key,year,,,,,,
08111,2004,0.002413,0.213166,0.784421,40680.228,70877.0,58.340000
08115,2004,0.004872,0.422414,0.572714,14478.332,39520.0,47.110001
08116,2004,0.006507,0.376005,0.617488,15205.272,30190.0,42.700001
08117,2004,0.013096,0.400742,0.586162,6158.588,24134.0,38.419998
08118,2004,0.014722,0.341771,0.643507,15851.325,31422.0,47.139999
...,...,...,...,...,...,...,...
16075,2012,0.044653,0.428515,0.526831,1955.848,23051.0,NaN
16076,2012,0.037358,0.373862,0.588780,1854.354,17851.0,NaN
16077,2012,0.028015,0.338902,0.633083,1689.681,17732.0,NaN


<a class="anchor" id="8. Difference in differences implementation"></a>

---
# gdp


##  gdp - no



In [9]:
exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_level,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_level,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_inhabitant,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_inhabitant,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_employee,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_employee,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.gdp_per_hour_per_employed,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.gdp_per_hour_per_employed,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))


print(compare({"(7)": res_fe_7, "(8)": res_fe_8}, stars = True, precision = "std_errors"))



                                               Model Comparison                                              
                                     (1)              (2)                       (3)                       (4)
-------------------------------------------------------------------------------------------------------------
Dep. Variable               ln_gdp_level     ln_gdp_level     ln_gdp_per_inhabitant     ln_gdp_per_inhabitant
Estimator                       PanelOLS         PanelOLS                  PanelOLS                  PanelOLS
No. Observations                    3573             3573                      3573                      3573
Cov. Est.                      Clustered        Clustered                 Clustered                 Clustered
R-squared                         0.1762           0.0385                    0.2973                    0.0501
R-Squared (Within)                0.4069           0.1023                    0.4699                    0.0384
R-Squared 

In [10]:
# gdp growth

exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp,exog,entity_effects=False)
res_fe_9 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp,exog,entity_effects=True,time_effects=True)
res_fe_10 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(9)": res_fe_9, "(10)": res_fe_10}, stars = True, precision = "std_errors"))

                   Model Comparison                   
                                    (9)           (10)
------------------------------------------------------
Dep. Variable                    ln_gdp         ln_gdp
Estimator                      PanelOLS       PanelOLS
No. Observations                   3573           3573
Cov. Est.                     Clustered      Clustered
R-squared                        0.0120         0.0032
R-Squared (Within)               0.0051         0.0007
R-Squared (Between)              0.1387        -4.3836
R-Squared (Overall)              0.0120        -0.2268
F-statistic                      7.2441         2.5131
P-value (F-stat)                 0.0000         0.0398
=====================     =============   ============
const                           -2708.0      1.233e+04
                            (1.046e+04)       (9219.3)
affected_after_no                0.5660         0.4207
                               (0.3752)       (0.3066)
year_post 

##  gdp - toas

In [11]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_level,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_level,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_inhabitant,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_inhabitant,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_employee,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_employee,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.gdp_per_hour_per_employed,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.gdp_per_hour_per_employed,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))

print(compare({"(7)": res_fe_7, "(8)": res_fe_8}, stars = True, precision = "std_errors"))




                                               Model Comparison                                               
                                      (1)              (2)                       (3)                       (4)
--------------------------------------------------------------------------------------------------------------
Dep. Variable                ln_gdp_level     ln_gdp_level     ln_gdp_per_inhabitant     ln_gdp_per_inhabitant
Estimator                        PanelOLS         PanelOLS                  PanelOLS                  PanelOLS
No. Observations                     3573             3573                      3573                      3573
Cov. Est.                       Clustered        Clustered                 Clustered                 Clustered
R-squared                          0.2511           0.0361                    0.3113                    0.0502
R-Squared (Within)                 0.4253           0.0997                    0.4705                    0.0165
R

In [12]:
# gdp growth

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp,exog,entity_effects=False)
res_fe_9 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp,exog,entity_effects=True,time_effects=True)
res_fe_10 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(9)": res_fe_9, "(10)": res_fe_10}, stars = True, precision = "std_errors"))

                   Model Comparison                   
                                    (9)           (10)
------------------------------------------------------
Dep. Variable                    ln_gdp         ln_gdp
Estimator                      PanelOLS       PanelOLS
No. Observations                   3573           3573
Cov. Est.                     Clustered      Clustered
R-squared                        0.0113         0.0026
R-Squared (Within)               0.0043         0.0045
R-Squared (Between)              0.1397        -5.7750
R-Squared (Overall)              0.0113        -0.2953
F-statistic                      6.7984         2.0696
P-value (F-stat)                 0.0000         0.0822
======================     ============   ============
const                           -2297.6      1.261e+04
                             (1.04e+04)       (9228.2)
affected_after_toas              0.0073        -0.1423
                               (0.3180)       (0.2563)
year_post 

<a class="anchor" id="8. Difference in differences implementation"></a>

---
# gva

##  gva - no



In [13]:
# 缺gva_level

# exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_agr_weight","emp_man_weight","emp_ser_weight"]
# exog = sm.add_constant(df_fe[exog_vars])
# mod = PanelOLS(df_fe.ln_gva_level,exog,entity_effects=False)
# res_fe_1 = mod.fit(cov_type = 'clustered')

# exog_vars = ['affected_after_no',"emp_agr_weight","emp_man_weight","emp_ser_weight"]
# exog = sm.add_constant(df_fe[exog_vars])
# mod = PanelOLS(df_fe.ln_gva_level,exog,entity_effects=True,time_effects=True)
# res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))


print(compare({"(7)": res_fe_7, "(8)": res_fe_8}, stars = True, precision = "std_errors"))




                   Model Comparison                  
                                   (3)            (4)
-----------------------------------------------------
Dep. Variable               ln_gva_agr     ln_gva_agr
Estimator                     PanelOLS       PanelOLS
No. Observations                  3555           3555
Cov. Est.                    Clustered      Clustered
R-squared                       0.6834         0.0490
R-Squared (Within)             -0.1536        -0.0268
R-Squared (Between)             0.7016         0.0612
R-Squared (Overall)             0.6834         0.0593
F-statistic                     1093.9         32.416
P-value (F-stat)                0.0000         0.0000
=====================     ============   ============
const                           1462.7         172.96
                              (1824.5)       (407.45)
affected_after_no               0.0169      -0.0268**
                              (0.0523)       (0.0106)
year_post                   

In [14]:
exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_fin,exog,entity_effects=False)
res_fe_9 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_10 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_trade,exog,entity_effects=False)
res_fe_11 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_12 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_public,exog,entity_effects=False)
res_fe_13 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_14 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(9)": res_fe_9, "(10)": res_fe_10, "(11)": res_fe_11, "(12)": res_fe_12}, stars = True, precision = "std_errors"))

print(compare({"(13)": res_fe_13, "(14)": res_fe_14}, stars = True, precision = "std_errors"))


# gva - no -significant agr  man fin  trade 

                                            Model Comparison                                           
                                       (9)               (10)                 (11)                 (12)
-------------------------------------------------------------------------------------------------------
Dep. Variable               ln_gva_ser_fin     ln_gva_ser_fin     ln_gva_ser_trade     ln_gva_ser_trade
Estimator                         PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                      3573               3573                 3573                 3573
Cov. Est.                        Clustered          Clustered            Clustered            Clustered
R-squared                           0.8072             0.0321               0.8062               0.1827
R-Squared (Within)                  0.2807             0.0650               0.2786               0.1001
R-Squared (Between)                 0.8130             0.3821   

##  gva - toas

In [15]:
# 缺gva_level

# exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_agr_weight","emp_man_weight","emp_ser_weight"]
# exog = sm.add_constant(df_fe[exog_vars])
# mod = PanelOLS(df_fe.ln_gva_level,exog,entity_effects=False)
# res_fe_1 = mod.fit(cov_type = 'clustered')

# exog_vars = ['affected_after_no',"emp_agr_weight","emp_man_weight","emp_ser_weight"]
# exog = sm.add_constant(df_fe[exog_vars])
# mod = PanelOLS(df_fe.ln_gva_level,exog,entity_effects=True,time_effects=True)
# res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))


print(compare({"(7)": res_fe_7, "(8)": res_fe_8}, stars = True, precision = "std_errors"))





                   Model Comparison                   
                                    (3)            (4)
------------------------------------------------------
Dep. Variable                ln_gva_agr     ln_gva_agr
Estimator                      PanelOLS       PanelOLS
No. Observations                   3555           3555
Cov. Est.                     Clustered      Clustered
R-squared                        0.6884         0.0472
R-Squared (Within)              -0.1457        -0.0206
R-Squared (Between)              0.7065         0.0577
R-Squared (Overall)              0.6884         0.0560
F-statistic                      1119.4         31.171
P-value (F-stat)                 0.0000         0.0000
======================     ============   ============
const                            1314.6         161.58
                               (1795.2)       (404.81)
affected_after_toas              0.0184        -0.0041
                               (0.0475)       (0.0096)
year_post 

In [16]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_fin,exog,entity_effects=False)
res_fe_9 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_10 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_trade,exog,entity_effects=False)
res_fe_11 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_12 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_public,exog,entity_effects=False)
res_fe_13 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_agr_weight","emp_man_weight","emp_ser_weight","ln_res"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_14 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(9)": res_fe_9, "(10)": res_fe_10, "(11)": res_fe_11, "(12)": res_fe_12}, stars = True, precision = "std_errors"))

print(compare({"(13)": res_fe_13, "(14)": res_fe_14}, stars = True, precision = "std_errors"))


#  gva - toas  -significant  man ser  fin  trade  pubilc 

                                            Model Comparison                                            
                                        (9)               (10)                 (11)                 (12)
--------------------------------------------------------------------------------------------------------
Dep. Variable                ln_gva_ser_fin     ln_gva_ser_fin     ln_gva_ser_trade     ln_gva_ser_trade
Estimator                          PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                       3573               3573                 3573                 3573
Cov. Est.                         Clustered          Clustered            Clustered            Clustered
R-squared                            0.8131             0.0354               0.8126               0.1854
R-Squared (Within)                   0.2881             0.0344               0.2809               0.1259
R-Squared (Between)                  0.8188            

<a class="anchor" id="8. Difference in differences implementation"></a>

---
# employee

## employee no

In [17]:
df_fe[['employee', 'emp_agr', 'emp_man','emp_ser', 'emp_ser_fin', 'emp_ser_trade', 'emp_ser_public',
       'ln_emp', 'ln_emp_ser_fin','ln_emp_agr', 'ln_emp_man', 'ln_emp_ser',
       'ln_emp_ser_trade', 'ln_emp_ser_public']]

,,employee,emp_agr,emp_man,emp_ser,emp_ser_fin,emp_ser_trade,emp_ser_public,ln_emp,ln_emp_ser_fin,ln_emp_agr,ln_emp_man,ln_emp_ser,ln_emp_ser_trade,ln_emp_ser_public
Regional_key,year,,,,,,,,,,,,,,
08111,2004,474.066986,1.144,101.055,371.868,127.722,106.565,137.581,13.069104,11.757611,7.042286,11.523420,12.826294,11.576510,11.831968
08115,2004,213.466003,1.040,90.171,122.255,32.859,52.878,36.518,12.271233,10.399981,6.946976,11.409463,11.713864,10.875743,10.505561
08116,2004,248.658997,1.618,93.497,153.544,34.541,66.856,52.147,12.423838,10.449902,7.388946,11.445685,11.941742,11.110296,10.861822
08117,2004,112.932999,1.479,45.257,66.197,12.366,25.701,28.130,11.634550,9.422706,7.299121,10.720113,11.100390,10.154285,10.244592
08118,2004,235.968002,3.474,80.647,151.847,40.387,58.357,53.103,12.371451,10.606263,8.153062,11.297837,11.930629,10.974335,10.879989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16075,2012,40.736000,1.819,17.456,21.461,4.408,6.981,10.072,10.614867,8.391176,7.506042,9.767439,9.973993,8.850947,9.217515
16076,2012,39.643002,1.481,14.821,23.341,3.864,9.603,9.874,10.587669,8.259458,7.300473,9.603800,10.057967,9.169831,9.197660
16077,2012,37.409000,1.048,12.678,23.683,3.859,9.211,10.613,10.529667,8.258163,6.954639,9.447623,10.072513,9.128154,9.269835


In [34]:

exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_agr_weight","emp_man_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_no','ln_res',"emp_agr_weight","emp_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"emp_agr_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"emp_agr_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))




                                  Model Comparison                                 
                                   (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------
Dep. Variable                   ln_emp         ln_emp     ln_emp_agr     ln_emp_agr
Estimator                     PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                  3573           3573           3573           3573
Cov. Est.                    Clustered      Clustered      Clustered      Clustered
R-squared                       0.8924         0.3174         0.7631         0.2166
R-Squared (Within)              0.3756        -0.1115        -0.2452         0.3256
R-Squared (Between)             0.8934         0.8030         0.7679         0.4317
R-Squared (Overall)             0.8924         0.8012         0.7631         0.4312
F-statistic                     4929.1         367.84         2297.8        

In [19]:
exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"emp_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"emp_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_5, "(2)": res_fe_6, "(3)": res_fe_7, "(4)": res_fe_8}, stars = True, precision = "std_errors"))

                                  Model Comparison                                 
                                   (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------
Dep. Variable               ln_emp_man     ln_emp_man     ln_emp_ser     ln_emp_ser
Estimator                     PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                  3573           3573           3573           3573
Cov. Est.                    Clustered      Clustered      Clustered      Clustered
R-squared                       0.8766         0.6450         0.8837         0.4844
R-Squared (Within)              0.4749         0.3159         0.5216         0.1994
R-Squared (Between)             0.8779         0.7004         0.8847         0.8017
R-Squared (Overall)             0.8766         0.6991         0.8837         0.8002
F-statistic                     5066.1         1917.0         5422.9        

In [20]:
exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_ser_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_fin,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_no','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_trade,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_public,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))

# empl no significant: man and ser (fin trade public)


                                            Model Comparison                                           
                                       (1)                (2)                  (3)                  (4)
-------------------------------------------------------------------------------------------------------
Dep. Variable               ln_emp_ser_fin     ln_emp_ser_fin     ln_emp_ser_trade     ln_emp_ser_trade
Estimator                         PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                      3573               3573                 3573                 3573
Cov. Est.                        Clustered          Clustered            Clustered            Clustered
R-squared                           0.8019             0.1182               0.8706               0.1708
R-Squared (Within)                  0.3847             0.1330               0.1121               0.0736
R-Squared (Between)                 0.8061             0.4946   

## employee toas

In [21]:
df_fe.columns

Index(['Gebietseinheit', 'EU_Code', 'Land', 'resident', 'ln_gdp', 'gdp_level',
       'gdp_per_inhabitant', 'gdp_per_employee', 'gdp_per_hour_per_employed',
       'ln_gva', 'gva_agr', 'gva_man', 'gva_ser', 'gva_ser_fin',
       'gva_ser_trade', 'gva_ser_public', 'employee', 'emp_agr', 'emp_man',
       'emp_ser', 'emp_ser_fin', 'emp_ser_trade', 'emp_ser_public',
       'volume_of_work', 'vow_agr', 'vow_man', 'vow_ser', 'vow_ser_fin',
       'vow_ser_trade', 'vow_ser_public', 'firms_no', 'firms_aff_no',
       'No_ratio', 'toas_sum_county', 'toas_aff_sum_county', 'toas_ratio',
       'emp_agr_weight', 'emp_man_weight', 'emp_ser_weight', 'gva_agr_weight',
       'gva_man_weight', 'gva_ser_weight', 'vow_agr_weight', 'vow_man_weight',
       'vow_ser_weight', 'county_affected_no', 'county_affected_toas',
       'year_post', 'affected_after_no', 'affected_after_toas', 'ln_res',
       'ln_gdp_level', 'ln_gdp_per_inhabitant', 'ln_gdp_per_employee',
       'ln_gva_agr', 'ln_gva_man', 'ln_gva

In [22]:
df_fe[['volume_of_work',
       'vow_agr', 'vow_man', 'vow_ser', 'vow_ser_fin','vow_ser_trade', 'vow_ser_public',]]

,,volume_of_work,vow_agr,vow_man,vow_ser,vow_ser_fin,vow_ser_trade,vow_ser_public
Regional_key,year,,,,,,,
08111,2004,697.348999,1.935,156.006,539.408,187.321,161.171,190.916
08115,2004,307.300995,2.102,137.127,168.071,44.687,77.277,46.107
08116,2004,356.113007,2.966,141.381,211.767,47.858,97.342,66.568
08117,2004,160.294006,3.153,68.098,89.043,17.513,35.963,35.568
08118,2004,336.239014,6.383,121.486,208.370,55.567,85.251,67.552
...,...,...,...,...,...,...,...,...
16075,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16076,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16077,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
exog_vars = ['affected_after_toas','year_post',"emp_agr_weight","emp_man_weight","emp_ser_weight","county_affected_toas",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_toas','ln_res',"emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"emp_agr_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"emp_agr_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))





                                  Model Comparison                                 
                                   (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------
Dep. Variable                   ln_emp         ln_emp     ln_emp_agr     ln_emp_agr
Estimator                     PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                  3573           3573           3573           3573
Cov. Est.                    Clustered      Clustered      Clustered      Clustered
R-squared                       0.8955         0.3215         0.7634         0.2221
R-Squared (Within)              0.3798        -0.1714        -0.2617         0.3378
R-Squared (Between)             0.8965         0.8103         0.7683         0.4462
R-Squared (Overall)             0.8955         0.8083         0.7634         0.4457
F-statistic                     4365.4         299.75         2301.4        

In [38]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"emp_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"emp_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_5, "(2)": res_fe_6, "(3)": res_fe_7, "(4)": res_fe_8}, stars = True, precision = "std_errors"))

                                  Model Comparison                                  
                                    (1)            (2)            (3)            (4)
------------------------------------------------------------------------------------
Dep. Variable                ln_emp_man     ln_emp_man     ln_emp_ser     ln_emp_ser
Estimator                      PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                   3573           3573           3573           3573
Cov. Est.                     Clustered      Clustered      Clustered      Clustered
R-squared                        0.8840         0.6444         0.8904         0.4845
R-Squared (Within)               0.4812         0.2706         0.5318         0.1514
R-Squared (Between)              0.8853         0.7137         0.8914         0.8082
R-Squared (Overall)              0.8840         0.7122         0.8904         0.8065
F-statistic                      5436.3         1912.1         57

In [37]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_ser_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_fin,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_toas','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_trade,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_public,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))


# empl toas significant total empl agr  man ser (fin public )




                                            Model Comparison                                            
                                        (1)                (2)                  (3)                  (4)
--------------------------------------------------------------------------------------------------------
Dep. Variable                ln_emp_ser_fin     ln_emp_ser_fin     ln_emp_ser_trade     ln_emp_ser_trade
Estimator                          PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                       3573               3573                 3573                 3573
Cov. Est.                         Clustered          Clustered            Clustered            Clustered
R-squared                            0.8094             0.1259               0.8804               0.1696
R-Squared (Within)                   0.3963             0.0833               0.1204               0.0502
R-Squared (Between)                  0.8135            

<a class="anchor" id="8. Difference in differences implementation"></a>

---
# volume_of_work

## volume_of_work no of firms


In [25]:
df_fe[['volume_of_work',
       'vow_agr', 'vow_man', 'vow_ser', 'vow_ser_fin','vow_ser_trade', 'vow_ser_public',
       'ln_vow', 
       'ln_vow_ser_fin', 'ln_vow_ser_trade','ln_vow_ser_public', 'ln_vow_agr', 'ln_vow_man', 'ln_vow_ser'
      ]]

,,volume_of_work,vow_agr,vow_man,vow_ser,vow_ser_fin,vow_ser_trade,vow_ser_public,ln_vow,ln_vow_ser_fin,ln_vow_ser_trade,ln_vow_ser_public,ln_vow_agr,ln_vow_man,ln_vow_ser
Regional_key,year,,,,,,,,,,,,,,
08111,2004,697.348999,1.935,156.006,539.408,187.321,161.171,190.916,20.362797,19.048334,18.897976,19.067344,14.475618,18.865405,20.105983
08115,2004,307.300995,2.102,137.127,168.071,44.687,77.277,46.107,19.543338,17.615193,18.162907,17.646475,14.558400,18.736418,18.939897
08116,2004,356.113007,2.966,141.381,211.767,47.858,97.342,66.568,19.690759,17.683749,18.393741,18.013735,14.902725,18.766969,19.170997
08117,2004,160.294006,3.153,68.098,89.043,17.513,35.963,35.568,18.892520,16.678454,17.398001,17.386957,14.963865,18.036458,18.304630
08118,2004,336.239014,6.383,121.486,208.370,55.567,85.251,67.552,19.633333,17.833100,18.261110,18.028408,15.669149,18.615310,19.154826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16075,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16076,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16077,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# data[]"vow_agr_weight","vow_man_weight","vow_ser_weight"

In [27]:

exog_vars = ['affected_after_no','year_post',"county_affected_no","vow_agr_weight","vow_man_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_no','ln_res',"vow_agr_weight","vow_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"vow_agr_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"vow_agr_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))



                                  Model Comparison                                 
                                   (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------
Dep. Variable                   ln_vow         ln_vow     ln_vow_agr     ln_vow_agr
Estimator                     PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                  3384           3384           3384           3384
Cov. Est.                    Clustered      Clustered      Clustered      Clustered
R-squared                       0.8822         0.3084         0.7493         0.2170
R-Squared (Within)              0.1648        -0.0460        -0.3303         0.3624
R-Squared (Between)             0.8834         0.8208         0.7549         0.4266
R-Squared (Overall)             0.8822         0.8193         0.7493         0.4263
F-statistic                     4213.4         334.03         2019.0        

In [28]:
exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"vow_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"vow_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_5, "(2)": res_fe_6, "(3)": res_fe_7, "(4)": res_fe_8}, stars = True, precision = "std_errors"))




                                  Model Comparison                                 
                                   (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------
Dep. Variable               ln_vow_man     ln_vow_man     ln_vow_ser     ln_vow_ser
Estimator                     PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                  3384           3384           3384           3384
Cov. Est.                    Clustered      Clustered      Clustered      Clustered
R-squared                       0.8739         0.6321         0.8856         0.5038
R-Squared (Within)              0.4620         0.4646         0.4265         0.3352
R-Squared (Between)             0.8755         0.7446         0.8867         0.8309
R-Squared (Overall)             0.8739         0.7435         0.8856         0.8297
F-statistic                     4682.0         1716.2         5228.0        

In [29]:

exog_vars = ['affected_after_no','year_post',"county_affected_no","vow_ser_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_fin,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_no','ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_trade,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_public,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))





#  vow no significant man ser (trade public )



                                            Model Comparison                                           
                                       (1)                (2)                  (3)                  (4)
-------------------------------------------------------------------------------------------------------
Dep. Variable               ln_vow_ser_fin     ln_vow_ser_fin     ln_vow_ser_trade     ln_vow_ser_trade
Estimator                         PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                      3384               3384                 3384                 3384
Cov. Est.                        Clustered          Clustered            Clustered            Clustered
R-squared                           0.8091             0.1439               0.8722               0.2006
R-Squared (Within)                  0.3423             0.1869               0.0676               0.1616
R-Squared (Between)                 0.8139             0.5964   

## volume_of_work toas

In [30]:

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","vow_agr_weight","vow_man_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_toas','ln_res',"vow_agr_weight","vow_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"vow_agr_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"vow_agr_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))



                                  Model Comparison                                  
                                    (1)            (2)            (3)            (4)
------------------------------------------------------------------------------------
Dep. Variable                    ln_vow         ln_vow     ln_vow_agr     ln_vow_agr
Estimator                      PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                   3384           3384           3384           3384
Cov. Est.                     Clustered      Clustered      Clustered      Clustered
R-squared                        0.8855         0.3140         0.7492         0.2271
R-Squared (Within)               0.1880        -0.0939        -0.3429         0.3970
R-Squared (Between)              0.8867         0.8269         0.7549         0.4317
R-Squared (Overall)              0.8855         0.8253         0.7492         0.4315
F-statistic                      4353.8         342.88         20

In [31]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"vow_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"vow_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_5, "(2)": res_fe_6, "(3)": res_fe_7, "(4)": res_fe_8}, stars = True, precision = "std_errors"))





                                  Model Comparison                                  
                                    (1)            (2)            (3)            (4)
------------------------------------------------------------------------------------
Dep. Variable                ln_vow_man     ln_vow_man     ln_vow_ser     ln_vow_ser
Estimator                      PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                   3384           3384           3384           3384
Cov. Est.                     Clustered      Clustered      Clustered      Clustered
R-squared                        0.8804         0.6321         0.8912         0.5050
R-Squared (Within)               0.4667         0.4348         0.4375         0.2994
R-Squared (Between)              0.8820         0.7583         0.8924         0.8370
R-Squared (Overall)              0.8804         0.7570         0.8912         0.8357
F-statistic                      4972.8         1716.2         55

In [32]:

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","vow_ser_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_fin,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_toas','ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_trade,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_public,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))





# vow toas signficant total vow agr ser (fin trade public )



                                            Model Comparison                                            
                                        (1)                (2)                  (3)                  (4)
--------------------------------------------------------------------------------------------------------
Dep. Variable                ln_vow_ser_fin     ln_vow_ser_fin     ln_vow_ser_trade     ln_vow_ser_trade
Estimator                          PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                       3384               3384                 3384                 3384
Cov. Est.                         Clustered          Clustered            Clustered            Clustered
R-squared                            0.8157             0.1481               0.8808               0.2023
R-Squared (Within)                   0.3504             0.1551               0.0810               0.1582
R-Squared (Between)                  0.8205            

In [33]:
# vow and empl by toas better 
# gva soso
# gdp not ok 

done
